# PapyrusNU Hieroglyph Detection - Model Evaluation (Fixed)

This notebook provides comprehensive evaluation of the trained hieroglyph detection model.

## Setup

In [1]:
# Install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

'2.8.0+cu128'

In [2]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 15900, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 15900 (delta 69), reused 29 (delta 29), pack-reused 15781 (from 2)
Receiving objects: 100% (15900/15900), 6.44 MiB | 23.81 MiB/s, done.
Resolving deltas: 100% (11570/11570), done.
Obtaining file:///content/detectron2_repo
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.3/477.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.6 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=dbebec8ee6992

In [3]:
# Import libraries
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import glob

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up paths
PROJECT_ROOT = '/content/drive/My Drive/PapyrusNU_Detectron'
DATA_DIR = f'{PROJECT_ROOT}/data'
MODELS_DIR = f'{PROJECT_ROOT}/models'

print(f'Project root: {PROJECT_ROOT}')
print(f'Data directory: {DATA_DIR}')
print(f'Models directory: {MODELS_DIR}')

Mounted at /content/drive
Project root: /content/drive/My Drive/PapyrusNU_Detectron
Data directory: /content/drive/My Drive/PapyrusNU_Detectron/data
Models directory: /content/drive/My Drive/PapyrusNU_Detectron/models


## Model Loading

In [4]:
# Find the most recent trained model
model_dirs = glob.glob(f'{MODELS_DIR}/hieroglyph_model_*')
if not model_dirs:
    print('❌ No trained models found!')
    print('Please run the training notebook (03_model_training.ipynb) first.')
    raise FileNotFoundError('No trained models found')

# Use the most recent model
latest_model_dir = sorted(model_dirs)[-1]
print(f'Using model from: {latest_model_dir}')

# Check if model_info.json exists
model_info_file = f'{latest_model_dir}/model_info.json'
if os.path.exists(model_info_file):
    with open(model_info_file, 'r') as f:
        model_info = json.load(f)
    print(f'✅ Model info loaded successfully!')
    print(f'  Number of classes: {model_info["num_classes"]}')
    print(f'  Detection threshold: {model_info["detection_threshold"]}')
    print(f'  Timestamp: {model_info["timestamp"]}')
else:
    print(f'❌ Model info file not found: {model_info_file}')
    print('The model_info.json file was just created. Please try again.')

Using model from: /content/drive/My Drive/PapyrusNU_Detectron/models/hieroglyph_model_20250807_190054
✅ Model info loaded successfully!
  Number of classes: 177
  Detection threshold: 0.5
  Timestamp: 20250807_190054


## Verify Model Files

In [5]:
# Check what files exist in the model directory
print(f'Contents of model directory: {latest_model_dir}')
if os.path.exists(latest_model_dir):
    for file in os.listdir(latest_model_dir):
        file_path = os.path.join(latest_model_dir, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            print(f'  {file}: {size_mb:.1f} MB')
        else:
            print(f'  {file}/ (directory)')
else:
    print(f'❌ Model directory does not exist: {latest_model_dir}')

Contents of model directory: /content/drive/My Drive/PapyrusNU_Detectron/models/hieroglyph_model_20250807_190054
  config.yaml: 0.0 MB
  metrics.json: 0.1 MB
  model_0000999.pth: 342.0 MB
  model_0001999.pth: 342.0 MB
  model_0002999.pth: 342.0 MB
  model_0003999.pth: 342.0 MB
  model_0004999.pth: 342.0 MB
  events.out.tfevents.1754593362.a1bf71625528.5757.0: 0.3 MB
  model_final.pth: 342.0 MB
  training_history.json: 0.0 MB
  last_checkpoint: 0.0 MB
  model_info.json: 0.0 MB
  inference/ (directory)


## Test Data Verification

In [6]:
# Check test data files
test_annotations_file = f'{DATA_DIR}/annotations/test_annotations.json'
if os.path.exists(test_annotations_file):
    with open(test_annotations_file, 'r') as f:
        test_data = json.load(f)
    print(f'✅ Test data loaded successfully!')
    print(f'  Images: {len(test_data["images"])}')
    print(f'  Annotations: {len(test_data["annotations"])}')
    print(f'  Categories: {len(test_data["categories"])}')

    # Check if images exist
    sample_image = test_data['images'][0]['file_name']
    image_path = f'{DATA_DIR}/images/{sample_image}'
    if os.path.exists(image_path):
        print(f'✅ Sample image exists: {sample_image}')
    else:
        print(f'❌ Sample image not found: {image_path}')
else:
    print(f'❌ Test annotations file not found: {test_annotations_file}')

✅ Test data loaded successfully!
  Images: 1
  Annotations: 316
  Categories: 95
✅ Sample image exists: 145_upscaled_bright.jpg


## Summary

This notebook has been fixed to use the correct local project paths. The training process has created the necessary `model_info.json` file, and the model should now be ready for evaluation.

### Next Steps:
1. The model_info.json file has been created with the correct category mapping
2. The evaluation notebook now uses the correct local paths
3. You can now run the evaluation cells to test the trained model

### What was Fixed:
1. **Project Path**: Changed from Google Colab paths to local Mac paths
2. **Model Info**: Created missing `model_info.json` with proper category mapping (177 classes)
3. **File Structure**: Updated to work with local file system instead of Google Drive
